In [38]:
import pyodbc
import pandas as pd
import datetime as d
import seaborn as sns
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [39]:
conn = pyodbc.connect("DRIVER={{SQL Server}};SERVER={0}; database={1};trusted_connection=yes;UID={2};PWD={3}".format('PETER','CO2DB','sa','peter1490'))


In [40]:
#we get max and min year form our db
#max year
max_year = pd.read_sql_query("EXECUTE GetWidgetsData 0,''", conn)
end_yaer = max_year['max_year'].values[0]

#Min Year
min_year = pd.read_sql_query("EXECUTE GetWidgetsData 1,''", conn)
start_year = min_year['min_year'].values[0]

#we create a year slider based on the max and min years
year_slider = pn.widgets.IntSlider(name='Year slider', start=int(start_year), end=int(end_yaer), step=5, value=2020)
year_slider

IntSlider(end=2020, name='Year slider', start=1750, step=5, value=2020)

In [41]:
# We get all gases and their IDs
pd_gases=pd.read_sql_query("EXECUTE GetWidgetsData 2,''", conn)

#We convert our Dataframe to Dic
dic_list = pd.Series(pd_gases.Gas_Name.values,index =pd_gases.Gas_Name).to_dict()

#We load the Dic to our Select with IDs as values
gas_select = pn.widgets.Select(name='Gases', options=dic_list)
gas_select

Select(name='Gases', options={'carbon dioxide (CO2)': '...}, value='carbon dioxide (CO2)')

In [42]:
# We get all Sources of CO2 and their IDs
pd_CO2_Sources=pd.read_sql_query("EXECUTE GetWidgetsData 3,''", conn)

#We convert our Dataframe to Dic
dic_list = pd.Series(pd_CO2_Sources.Source_ID.values,index =pd_CO2_Sources.Sources_Name).to_dict()

#We load the Dic to our Select with IDs as values
sources_select = pn.widgets.Select(name='CO2 Source', options=dic_list)
sources_select

Select(name='CO2 Source', options={'Cement': 44, ...}, value=44)

In [58]:
# We get all Countries and their IDs
pd_Countries=pd.read_sql_query("EXECUTE GetWidgetsData 4,''", conn)

#We convert the countries into a list
Countries_list  = list(pd_Countries['Country_Name'])
Countries_select = pn.widgets.AutocompleteInput(
    name='Countries', 
    options=Countries_list,
    placeholder='Enter Name of the country',
value="United Kingdom")
Countries_select

AutocompleteInput(name='Countries', options=['Afghanistan', ...], placeholder='Enter Name of t..., value='United Kingdom')

In [59]:
pd_Country_stas=pd.read_sql_query("EXECUTE GetCountryGreenHouseGases 0,''", conn)


In [60]:
Ipd_Country_stas = pd_Country_stas.interactive()


In [61]:
Country_gas_emission_pipe = (Ipd_Country_stas[
    (Ipd_Country_stas.Year <= year_slider ) 
    & (Ipd_Country_stas.Country_Name == Countries_select )
   
]
                             .groupby(['Country_Name','Year','Gas_Name'])['Amount']
                             .sum()
                             .to_frame()
                             .reset_index()
                             .sort_values(by='Year')  
                             .reset_index(drop=True)

                            )

In [62]:
Gas_emission_table = Country_gas_emission_pipe.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
Gas_emission_table

In [63]:
Gas_Emission_bar_plot = Country_gas_emission_pipe.hvplot(kind='line', 
                                                     x='Year', 
                                                     y='Amount', 
                                                     title='Gas Emission',
                                                     by='Gas_Name',
                                                    )

In [64]:
Gas_Emission_bar_plot

In [65]:
pd_CO2_sources_Country = pd.read_sql_query("EXECUTE GetCO2ByDifferentSourcesPerCountry '3','1','Kenya'", conn)

In [66]:
IP_CO2_sources_Country = pd_CO2_sources_Country.interactive()

In [67]:
CO2_sources_Country_pipe = (IP_CO2_sources_Country[
    (Ipd_Country_stas.Year <= year_slider ) 
    & (Ipd_Country_stas.Country_Name == Countries_select )
   
]
                             .groupby(['Sources_Name','Year','Country_Name','Gas_Name'])['Amount']
                             .sum()
                             .to_frame()
                             .reset_index()
                             .sort_values(by='Year')  
                             .reset_index(drop=True)

                            )

In [68]:
CO2_sources_table = CO2_sources_Country_pipe.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
CO2_sources_table

In [69]:
CO2_sources_bar_plot = CO2_sources_Country_pipe.hvplot(kind='line', 
                                                     x='Year', 
                                                     y='Amount', 
                                                     title='Sources of CO2',
                                                     by='Sources_Name',
                                                    )

In [70]:
CO2_sources_bar_plot

In [71]:
country = Countries_select

In [72]:
pd_Country_greenhouse_gases = pd.read_sql_query("EXECUTE GetCountryGreenHouseGases 0,'' ", conn)

In [73]:
IP_pd_Country_greenhouse_gases = pd_Country_greenhouse_gases.interactive()

In [74]:
Country_greenhouse_gases_pipeline = (IP_pd_Country_greenhouse_gases[
    (Ipd_Country_stas.Year <= year_slider ) 
    & (Ipd_Country_stas.Country_Name == Countries_select )
   
]
                             .groupby(['Year','Country_Name','Gas_Name'])['Amount']
                             .sum()
                             .to_frame()
                             .reset_index()
                             .sort_values(by='Year')  
                             .reset_index(drop=True)

                            )

In [75]:
Country_greenhouse_gases_table = Country_greenhouse_gases_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
Country_greenhouse_gases_table

In [76]:
co2_source_bar_plot = Country_greenhouse_gases_pipeline.hvplot(kind='bar', 
                                                     x='Gas_Name', 
                                                     y='Amount', 
                                                     title='CO2 source by continent')

In [77]:
co2_source_bar_plot

In [79]:
template = pn.template.FastListTemplate(
    title='Countries\' Greenhouse Gases dashboard', 
    sidebar=[pn.pane.Markdown("# Greenhouse Gases and Climate Change"), 
             pn.pane.Markdown("#### Greenhouse gases emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
            
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[
            pn.Row(pn.Column(Countries_select, Gas_Emission_bar_plot.panel(width=700), margin=(0,25)), Gas_emission_table.panel(width=500)), 
            pn.Row(pn.Column(CO2_sources_bar_plot.panel(width=700), margin=(0,25)), CO2_sources_table.panel(width=500)), 
            pn.Row(pn.Column(co2_source_bar_plot.panel(width=700), margin=(0,25)), Country_greenhouse_gases_table.panel(width=500)), 
           # pn.Row(pn.Column(CO2_sources_bar_plot.panel(width=600), margin=(0,25)), pn.Column(co2_source_bar_plot.panel(width=600)))
    ],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();